# **Lidar cone detection**

## **Setup**

Setup the repo and the model.

### **Clone the repositories and import the libraries**

Clone the repository from the smart application course organization and the one for the yolov5 model and import the libraries needed to work with the porject.

In [1]:
!git clone --recurse-submodules -b dev_Petix_Ristori "https://github.com/unipi-smartapp-2021/lidar-cone-detection"   # clone our repository
!git clone "https://github.com/ultralytics/yolov5"  # clone the model repository

fatal: destination path 'lidar-cone-detection' already exists and is not an empty directory.
fatal: destination path 'yolov5' already exists and is not an empty directory.


Install the requirements.

In [ ]:
!pip install -r lidar-cone-detection/requirements.txt

Import the libraries and mount the drive.

In [4]:
from tqdm import tqdm
import tarfile
from google.colab import drive
drive.mount('/content/drive')  # mount the drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Manage the dataset**

If the dataset is inside a rosbag extract it, otherwise manage it with the manage_dataset.py method.

In [5]:
fname = "/content/drive/Shareddrives/Sensory_data/Recordings/2021-11-28-16-29-25 lidar+camera.tar.xz"
if fname.endswith("tar.xz"):
    tar = tarfile.open(fname, "r:xz")
    tar.extractall("lidar_dataset")
    tar.close()

Manage the dataset by following what is suggested in https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data chapter 1.3. Keep in mind to modify the directory from which you're moving the images (ConeDataset for the first dataset or TRset for the full one).

You can decide to convert the images to grayscale ones or not, then the dataset is splited into train, validation and test sets.

In [ ]:
!python lidar-cone-detection/src/manage_dataset.py --dataset ConeDataset --path lidar-cone-detection/src/datasets/ConeDataset

### **Check the yolov5 model and import the weights**

Check if yolov5 is ready and fine.

In [6]:
%cd yolov5
from yolov5.utils import notebook_init
display = notebook_init()  # checks
%cd ../

YOLOv5 🚀 v6.0-120-g92a7391 torch 1.10.0+cu111 CPU


Setup complete ✅
/content


Import the yolov5 model, all the available models can be found here: https://github.com/ultralytics/yolov5/releases/.

In [7]:
import requests
url = 'https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt'
r = requests.get(url, allow_redirects=True)
open('yolov5s.pt', 'wb').write(r.content)

14698491

## **Convert from rosbag to pcd**

If the dataset is inside a rosbag convert each message to a pcd file by calling the right method.

In [9]:
!python lidar-cone-detection/src/lidar/convert_rosbag_to_pcd.py --rosbag /content/lidar_dataset/2021-11-28-16-29-25.bag --path lidar-cone-detection/src/lidar/pcd_lidar

  0% 0/1800 [00:00<?, ?it/s]95961901796
  0% 0/1800 [00:00<?, ?it/s]


Save the pcd files inside a tar.gz by using the next command.

In [ ]:
!python -m tarfile -c lidar_dataset.tar.gz "lidar-cone-detection/src/Lidar/pcd_outputs/"

## **Train the model**

Train the model following the guide from https://github.com/ultralytics/yolov5/blob/master/tutorial.ipynb.

**Main arguments:**
*   *--img*, image size.
*   *--batch*, batch size.
*   *--epochs*, number of epochs.
*   *--data*, the yaml file that specifies where datasets are.
*   *--weights*, the model weights (you can find them here https://github.com/ultralytics/yolov5/releases).
*   *--cache*, to save a cache file of the train and validation sets.
*   *--project*, where to save the results.

**An example of train call would be:**

!python yolov5/train.py --img img_size --batch batch_size --epochs epochs --data dataset.yaml --weights model --cache --project runs/train








In [ ]:
!python yolov5/train.py --img 640 --batch 8 --epochs 100 --data /content/lidar-cone-detection/conedataset_lidar.yaml --weights yolov5s.pt --cache --project runs/train